In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vi_data_df = pd.read_csv("./data/vi_text_retrieval.csv")
context = vi_data_df["text"]
context = [doc.lower() for doc in context]

tfidf_vectorizer = TfidfVectorizer()
context_embedded = tfidf_vectorizer.fit_transform(context)
context_embedded.toarray()[7][0]

0.3112658076071063

In [ ]:
def tfidf_search(question, tfidf_vectorizer, top_d=5):
    query_embedded = tfidf_vectorizer.transform([question.lower()])
    cosine_scores = cosine_similarity(context_embedded, query_embedded).reshape((-1,))

    # Get top k cosine score and index its
    results = []
    for idx in cosine_scores.argsort()[-top_d:][::-1]:
        doc_score = {"id": idx, "cosine_score": cosine_scores[idx]}
        results.append(doc_score)
    return results


question = vi_data_df.iloc[0]["question"]
results = tfidf_search(question, tfidf_vectorizer, top_d=5)
results[0]["cosine_score"]

0.6279910475266973

In [ ]:
def corr_search(question, tfidf_vectorizer, top_d=5):
    query_embedded = tfidf_vectorizer.transform([question.lower()])
    corr_scores = np.corrcoef(query_embedded.toarray()[0], context_embedded.toarray())

    corr_scores = corr_scores[0][1:]
    results = []
    for idx in corr_scores.argsort()[-top_d:][::-1]:
        doc = {"id": idx, "corr_score": corr_scores[idx]}
        results.append(doc)

    return results


question = vi_data_df.iloc[0]["question"]
results = corr_search(question, tfidf_vectorizer, top_d=5)
results[1]["corr_score"]

0.20734246471973258

In [ ]:
# Plagiarism Checker
docs = [
    "Học máy là một nhánh của trí tuệ nhân tạo",
    "Trí tuệ nhân tạo bao gồm học máy và mạng rơ ron",
    "Mạng nơ ron là một mô hình quan trọng trong học sâu",
    "HỌc sâu là một lĩnh vực của trí tuệ nhân tạo và học máy",
]

In [ ]:
def build_vocabulary(docs):
    vocab_set = set()
    for doc in docs:
        words = doc.lower().split()
        vocab_set.update(words)
    vocab = sorted(list(vocab_set))
    return vocab


vocab = build_vocabulary(docs)
print(vocab)

['bao', 'của', 'gồm', 'hình', 'học', 'là', 'lĩnh', 'máy', 'mô', 'mạng', 'một', 'nhánh', 'nhân', 'nơ', 'quan', 'ron', 'rơ', 'sâu', 'trong', 'trí', 'trọng', 'tuệ', 'tạo', 'và', 'vực']


In [ ]:
def compute_tf(doc, vocab):
    words = doc.lower().split()
    tf = np.zeros(len(vocab))
    for i, term in enumerate(vocab):
        tf[i] = words.count(term) / len(words)
    return tf


for doc in docs:
    print(compute_tf(doc, vocab))

[0.  0.1 0.  0.  0.1 0.1 0.  0.1 0.  0.  0.1 0.1 0.1 0.  0.  0.  0.  0.
 0.  0.1 0.  0.1 0.1 0.  0. ]
[0.08333333 0.         0.08333333 0.         0.08333333 0.
 0.         0.08333333 0.         0.08333333 0.         0.
 0.08333333 0.         0.         0.08333333 0.08333333 0.
 0.         0.08333333 0.         0.08333333 0.08333333 0.08333333
 0.        ]
[0.         0.         0.         0.08333333 0.08333333 0.08333333
 0.         0.         0.08333333 0.08333333 0.08333333 0.
 0.         0.08333333 0.08333333 0.08333333 0.         0.08333333
 0.08333333 0.         0.08333333 0.         0.         0.
 0.        ]
[0.         0.07142857 0.         0.         0.14285714 0.07142857
 0.07142857 0.07142857 0.         0.         0.07142857 0.
 0.07142857 0.         0.         0.         0.         0.07142857
 0.         0.07142857 0.         0.07142857 0.07142857 0.07142857
 0.07142857]


In [ ]:
def compute_idf(docs, vocab):
    N = len(docs)
    idf = np.zeros(len(vocab))
    for i, term in enumerate(vocab):
        df = sum([1 for doc in docs if term in doc.lower().split()])
        idf[i] = np.log((N + 1) / (df + 1) + 1)
    return idf


idf = compute_idf(docs, vocab)
print(idf)

[1.25276297 0.98082925 1.25276297 1.25276297 0.69314718 0.81093022
 1.25276297 0.81093022 1.25276297 0.98082925 0.81093022 1.25276297
 0.81093022 1.25276297 1.25276297 0.98082925 1.25276297 0.98082925
 1.25276297 0.81093022 1.25276297 0.81093022 0.81093022 0.98082925
 1.25276297]


In [ ]:
def compute_tfidf(tf, idf):
    return tf * idf


for doc in docs:
    tf = compute_tf(doc, vocab)
    tf_idf = compute_tfidf(tf, idf)
    print(tf_idf)

[0.         0.09808293 0.         0.         0.06931472 0.08109302
 0.         0.08109302 0.         0.         0.08109302 0.1252763
 0.08109302 0.         0.         0.         0.         0.
 0.         0.08109302 0.         0.08109302 0.08109302 0.
 0.        ]
[0.10439691 0.         0.10439691 0.         0.05776227 0.
 0.         0.06757752 0.         0.08173577 0.         0.
 0.06757752 0.         0.         0.08173577 0.10439691 0.
 0.         0.06757752 0.         0.06757752 0.06757752 0.08173577
 0.        ]
[0.         0.         0.         0.10439691 0.05776227 0.06757752
 0.         0.         0.10439691 0.08173577 0.06757752 0.
 0.         0.10439691 0.10439691 0.08173577 0.         0.08173577
 0.10439691 0.         0.10439691 0.         0.         0.
 0.        ]
[0.         0.07005923 0.         0.         0.09902103 0.05792359
 0.08948307 0.05792359 0.         0.         0.05792359 0.
 0.05792359 0.         0.         0.         0.         0.07005923
 0.         0.0579235

In [29]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot_product / (norm1 * norm2)